Решение задач из курса [Нейронные сети и обработка текста](https://stepik.org/course/54098).

# Задача поиска
Процесс решения задачи поиска часто состоит из двух крупных шагов, каждый из которых разбивается на более мелкие:

1. Настройка поиска
    1. Преобразование объектов (например, текстов) в вещественные вектора
    2. Построение поискового индекса
    3. Настройка функции ранжирования
2. Выполнение поиска
    1. Преобразование запроса в вещественный вектор
    2. Грубая выборка кандидатов
    3. Сртировка кандидатов с помощью функции ранжирования

Поисковый индекс – набор специальных структур данных, ускоряющих процесс поиска. Так как процесс индексации тоже требует времени, поисковые индексы не всегда имеет смысл строить – например, когда данных мало или данные часто меняются и индекс устаревает быстрее, чем может быть перестроен.

Настройка функции ранжирования выполняется с помощью набора примеров вида "запрос – документ – оценка релевантности по мнению человека". Релевантность – численная величина, характеризующая соответствие найденного документа запросу (чем больше, тем лучше документ подходит под запрос).

Пусть у нас есть коллекция документов, которые на шаге 1.1 были преобразованы в следующие вектора

ID документа | Признаки
--- | ---
1 | (0, 1)
2 | (1, 0)
3 | (1, 0.5)

Также у нас есть функция ранжирования $Relevance(q, d) = -(q_1 - d_1)^2 - 2 (q_2 - d_2)^2$, где $(q_1, q_2)$ – признаки запроса, а $(d_1, d_2)$ признаки документа.

Нам пришел запрос с признаками (1, 1).

Определите релевантность документов относительно данного запроса.

In [1]:
def relevance(q, d):
    return -(q[0] - d[0])**2 - 2*((q[1] - d[1])**2)

docs = [(0.0, 1.0),
        (1.0, 0.0),
        (1.0, 0.5)]

q = (1, 1)

for doc in docs:
    print(f"Relevance for {doc} is {relevance(q, doc)}.")

Relevance for (0.0, 1.0) is -1.0.
Relevance for (1.0, 0.0) is -2.0.
Relevance for (1.0, 0.5) is -0.5.


# Обучение логистической регрессии. Градиентный спуск

**Задача**. Вы обучаете одномерную логистическую регрессию $\hat{y}(x) = \frac{1} {1 + e^{-wx - b}}$ , то есть $w \in R$ – это скаляр (число), $x$ –  единственный признак входного объекта, $y(x) \in \{0, 1\}$ – настоящий класс объекта $x$, $ 0 < \hat{y}(x) < 1$ – предсказанная вероятность того, что $x$ принадлежит к классу 1.

В качестве функции потерь вы используете бинарную кросс-энтропию $$BCE(\hat{y}, y) = -y \log\hat{y} - (1 - y) \log(1 - \hat{y})$$

Найдите в общем виде производную функции потерь по $w$ $\frac{\partial BCE(\hat{y}, y)}{\partial w}$.

In [2]:
from sympy import *
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
diff(eq, w).simplify()

x*(-y*exp(-b - w*x) - y + 1)/(exp(-b - w*x) + 1)

Теперь найдите в общем виде производную функции потерь по $b$ $\frac{\partial BCE(\hat{y}, y)}{\partial b}$.

In [3]:
diff(eq, b).simplify()

(-y*exp(-b - w*x) - y + 1)/(exp(-b - w*x) + 1)

Теперь добавьте L2-регуляризацию $Loss(\hat{y}, y) = BCE(\hat{y}, y) + c(w^2 + b^2)$, где $c > 0$ – коэффициент регуляризации. Расчитайте производную функции потерь по w.

In [4]:
c = symbols('c')
eq += c*(w*w + b*b)
diff(eq, w).simplify()

(2*c*w*(exp(-b - w*x) + 1) - x*y*exp(-b - w*x) - x*(y - 1))/(exp(-b - w*x) + 1)

Используя формулу производной с предыдущего шага, запишите формулу для обновления веса $w$ с помощью стохастического градиентного спуска (размер минибатча равен 1). Для обозначения скорости обучения (learning rate) используйте маленькую латинскую букву $t$.

In [5]:
t = symbols('t')
w_new = w - t*diff(eq, w).simplify()
w_new

-t*(2*c*w*(exp(-b - w*x) + 1) - x*y*exp(-b - w*x) - x*(y - 1))/(exp(-b - w*x) + 1) + w

# Точечная взаимная информация двух случайных событий

**Задача**. Напишите функцию для вычисления точечной взаимной информации двух случайных событий.

$pmi(a, b) = \ln \dfrac{p(a, b)} {p(a) p(b)}$

На вход функция получает два массива из 0 и 1 одинаковой длины – реализации случайных событий: 1 – событие произошло, 0 – не произошло. В результате функция должна вернуть вещественное число – точечную взаимную информацию событий.

```
Sample Input:
    1 0 0 1 1 0
    1 0 0 0 1 0

Sample Output:
    0.693147
```

In [6]:
import numpy as np
from collections import Counter
 
def calculate_pmi(a, b):
    def prob(arr):
        return Counter(arr)[1]/len(arr)

    p_ab = Counter(zip(a, b))[(1, 1)]/len(a)
    return np.log(p_ab/(prob(a)*prob(b)))

a = [1, 0, 0, 1, 1, 0]
b = [1, 0, 0, 0, 1, 0]
pmi_value = calculate_pmi(a, b)

print('{:.6f}'.format(pmi_value))

0.693147


**Задача**. Найдите количество слов, которые встречаются менее, чем в 10 из 10000 документов, если предполагать, что вероятность встретить слово в документе распределена по Ципфу с параметром $s = 2$, количество слов в словаре $N = 1000$. Ранги нумеруются с 1.

**Решение**. Плотность распределения Ципфа имеет следующий вид:

$$f(rank,s, N) = \dfrac{1}{Z(s, N)rank^s},$$

где $rank$ – порядковый номер слова после сортировки по убыванию частоты, $s$ – коэффициент скорости убывания вероятности, $N$ – количество слов, $Z(s, N) = \sum_{i=1}^N i^{-s}$ – нормализационная константа.

In [7]:
s = 2
N = 1000

def Z(s, N):
    return sum(i**-s for i in range(1, N+1))

print(f'Значение Z(s, N) для s = {s} и N = {N} составляет {Z(s, N):.2f}.')

Значение Z(s, N) для s = 2 и N = 1000 составляет 1.64.


Используем заданный в условии критерий отбора: $f_c < 0.001$ (т.е. 10/10000).

In [10]:
Z_selected = Z(s, N)
f_c = 0.001

for rank in range(1, 1000):
    f = 1/(Z_selected*rank**s)
    if f < f_c:
        break

# нам нужно предыдущее значение ранга
# перед тем как оно стало ниже критического
rank -= 1
print(N - rank)

976


**Задача**. Дана следующая коллекция текстов. Постройте словарь (отображение из строкового представления токенов в их номера) и вектор весов (DF).


```
Казнить нельзя, помиловать. Нельзя наказывать.

Казнить, нельзя помиловать. Нельзя освободить.

Нельзя не помиловать.

Обязательно освободить.

```

$DF(w) = \frac{DocCount(w, c)}{Size(c)}$ – частота слова $w$ в коллекции $c$ (отношение количества документов, в которых слово используется, к общему количеству документов).

In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

list_ = ['Казнить нельзя, помиловать. Нельзя наказывать.',
         'Казнить, нельзя помиловать. Нельзя освободить.',
         'Нельзя не помиловать.',
         'Обязательно освободить.']

vector = TfidfVectorizer(smooth_idf=False, use_idf=True)
vector.fit_transform(list_)
freq = 1/np.exp(vectorizer.idf_ - 1)

df = pd.DataFrame(zip(vector.get_feature_names(), freq)).sort_values(by=1)
print(' '.join(df[0].values))
print(' '.join([f'{i:.2f}' for i in df[1].values]))

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.50 0.50 0.75 0.75
